In [4]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [7]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets

In [8]:
audio = pd.read_csv('data/audio_features_hot_100_1958_2019.csv')
genre = pd.read_csv('data/songGenre.csv')[['track_id','supergenre']].rename(columns = {'songid':'SongID'})
genre = genre.drop_duplicates()
genre['supergenre'].unique()


array(['pop', 'country', 'rap', 'hiphop', 'other', 'latin', 'house',
       'folk', 'r&b', 'adult standards', 'rock', 'metal', 'show tunes',
       'soul', 'jazz'], dtype=object)

In [44]:
join_df = genre.merge(audio, on = 'track_id', how = 'left').dropna(how='any')
# target = join_df[['supergenre']]
feature_names =['spotify_track_duration_ms','danceability','energy','loudness','speechiness','acousticness','instrumentalness','valence']
features = join_df[feature_names]
features.head()

,spotify_track_duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,valence
0,270740.0,0.734,0.836,-4.803,0.0735,0.01700,0.000016,0.623
2,215733.0,0.572,0.385,-6.362,0.0308,0.37100,0.000000,0.323
3,215733.0,0.572,0.385,-6.362,0.0308,0.37100,0.000000,0.323
4,196760.0,0.542,0.674,-4.169,0.2100,0.05880,0.000000,0.667
5,228185.0,0.948,0.623,-5.725,0.1680,0.00124,0.000001,0.856


In [81]:
join_df['target'] = join_df['supergenre'].replace('pop',float(0)).replace('country',float(1))\
.replace('hiphop',float(2)).replace('other',float(3)).replace('latin',float(3)).replace('latin',float(4))\
.replace('house',float(5)).replace('folk',float(6)).replace('r&b',float(7)).replace('adult standards',float(8))\
.replace('rock',float(9)).replace('metal',float(10)).replace('show tunes',float(11)).replace('soul',float(12))\
.replace('rap',float(13)).replace('jazz',float(14))

target = join_df[['target']]

In [82]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1)
y_train.head()

,target
9035,9.0
11009,0.0
7782,9.0
9085,1.0
25491,5.0


In [83]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

In [84]:
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [85]:
from tensorflow.keras.utils import to_categorical

In [86]:
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [98]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=8))
model.add(Dense(units=15, activation='softmax'))

In [99]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 6)                 54        
_________________________________________________________________
dense_13 (Dense)             (None, 15)                105       
Total params: 159
Trainable params: 159
Non-trainable params: 0
_________________________________________________________________


In [100]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [102]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
17670/17670 - 1s - loss: 1.6683 - acc: 0.3962
Epoch 2/100
17670/17670 - 1s - loss: 1.6683 - acc: 0.3962
Epoch 3/100
17670/17670 - 1s - loss: 1.6679 - acc: 0.3935
Epoch 4/100
17670/17670 - 1s - loss: 1.6680 - acc: 0.3949
Epoch 5/100
17670/17670 - 1s - loss: 1.6677 - acc: 0.3959
Epoch 6/100
17670/17670 - 1s - loss: 1.6678 - acc: 0.3947
Epoch 7/100
17670/17670 - 2s - loss: 1.6678 - acc: 0.3956
Epoch 8/100
17670/17670 - 2s - loss: 1.6680 - acc: 0.3963
Epoch 9/100
17670/17670 - 2s - loss: 1.6679 - acc: 0.3960
Epoch 10/100
17670/17670 - 1s - loss: 1.6678 - acc: 0.3956
Epoch 11/100
17670/17670 - 2s - loss: 1.6674 - acc: 0.3967
Epoch 12/100
17670/17670 - 2s - loss: 1.6676 - acc: 0.3967
Epoch 13/100
17670/17670 - 2s - loss: 1.6675 - acc: 0.3965
Epoch 14/100
17670/17670 - 2s - loss: 1.6674 - acc: 0.3969
Epoch 15/100
17670/17670 - 1s - loss: 1.6674 - acc: 0.3962
Epoch 16/100
17670/17670 - 2s - loss: 1.6671 - acc: 0.3975
Epoch 17/100
17670/17670 - 2s - loss: 1.6675 - acc: 0.3975
Epoch 

In [129]:
deep_model = Sequential()
deep_model.add(Dense(units=20, activation='selu', input_dim=8)) #selu vs relu?
deep_model.add(Dense(units=25, activation='selu'))
deep_model.add(Dense(units=25, activation='selu'))
deep_model.add(Dense(units=15, activation='softmax'))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [130]:
deep_model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 20)                180       
_________________________________________________________________
dense_41 (Dense)             (None, 25)                525       
_________________________________________________________________
dense_42 (Dense)             (None, 25)                650       
_________________________________________________________________
dense_43 (Dense)             (None, 15)                390       
Total params: 1,745
Trainable params: 1,745
Non-trainable params: 0
_________________________________________________________________


In [131]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
17670/17670 - 1s - loss: 1.8873 - acc: 0.3544
Epoch 2/100
17670/17670 - 1s - loss: 1.7039 - acc: 0.3824
Epoch 3/100
17670/17670 - 1s - loss: 1.6850 - acc: 0.3886
Epoch 4/100
17670/17670 - 1s - loss: 1.6743 - acc: 0.3890
Epoch 5/100
17670/17670 - 1s - loss: 1.6643 - acc: 0.3930
Epoch 6/100
17670/17670 - 1s - loss: 1.6564 - acc: 0.3968
Epoch 7/100
17670/17670 - 1s - loss: 1.6494 - acc: 0.3993
Epoch 8/100
17670/17670 - 2s - loss: 1.6436 - acc: 0.3999
Epoch 9/100
17670/17670 - 2s - loss: 1.6397 - acc: 0.4001
Epoch 10/100
17670/17670 - 2s - loss: 1.6358 - acc: 0.4026
Epoch 11/100
17670/17670 - 2s - loss: 1.6320 - acc: 0.4062
Epoch 12/100
17670/17670 - 2s - loss: 1.6271 - acc: 0.4065
Epoch 13/100
17670/17670 - 2s - loss: 1.6246 - acc: 0.4097
Epoch 14/100
17670/17670 - 2s - loss: 1.6229 - acc: 0.4074
Epoch 15/100
17670/17670 - 1s - loss: 1.6197 - acc: 0.4073
Epoch 16/100
17670/17670 - 1s - loss: 1.6170 - acc: 0.4095
Epoch 17/100
17670/17670 - 1s - loss: 1.6154 - acc: 0.4123
Epoch 